<head>
  <meta name="author" content="João Alves">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right"> 

<h1 align=left><font size = 8, style="color:rgb(200,0,0)"><b>Ciência de Dados </b></font></h1> 
<h1 align=left><font size = 5, style="color:rgb(0,0,0)"><b>Coleta e Armazenamento de Dados</b></font></h1> 

# Trilha 6 - Tarefa de aprofundamento
Professor: *Fabio Lopes* 

Aluno: *João Alves*
<br>
<br>

---


# Introdução 

Agora já conhecemos os bancos de dados NoSQL e as suas características.
Vamos então trabalhar estes conceitos na nossa última atividade de aprofundamento.
Selecione um banco de dados NoSQL de sua preferência. Crie uma instância de teste e faça a inserção de dados a partir de um arquivo csv.
Utilize a base de dados World Happiness disponível no site do Kaggle: <https://www.kaggle.com/ajaypalsinghlo/world-happiness-report-2021>
Apresente um notebook com o código utilizado e uma evidência do cadastro realizado.

Obs.: Alguns bancos de dados possuem área de sandbox disponível em nuvem. Se você tiver alguma dificuldade em baixar o banco, utilize a opção em nuvem.
Exemplos:

- https://www.mongodb.com/online
- https://sandbox.neo4j.com/
- https://blog.couchbase.com/nosql-proof-concept-data-creation/

*Você deve encaminhar um arquivo contendo o código utilizado e a evidência de que a base de dados foi carregada no banco selecionado. Uma lista simples, apresentando algumas linhas cadastradas.*


# Instalando as bibliotecas python

In [183]:
!pip install neo4j-driver

# Importando as bibliotecas python

In [184]:
import os
import numpy as np
import pandas as pd
from neo4j import GraphDatabase, basic_auth

# Lendo Variáveis de Ambiente

In [185]:
# Neo4j Info
neo_username = os.getenv('USERNAME')
neo_password = os.getenv('PASSWORD')
neo_bolt_url = os.getenv('BOLT_URL')

## Exploração Inicial dos Dados

In [186]:
df_happiness = pd.read_csv('../data/world-happiness-report.csv')

display(df_happiness.shape)
display(df_happiness.head())
display(df_happiness.tail())

(1949, 11)

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
0,Afghanistan,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882,0.518,0.258
1,Afghanistan,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850,0.584,0.237
2,Afghanistan,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707,0.618,0.275
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731,0.611,0.267
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.710,0.268


,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
1944,Zimbabwe,2016,3.735,7.984,0.768,54.4,0.733,-0.095,0.724,0.738,0.209
1945,Zimbabwe,2017,3.638,8.016,0.754,55.0,0.753,-0.098,0.751,0.806,0.224
1946,Zimbabwe,2018,3.616,8.049,0.775,55.6,0.763,-0.068,0.844,0.710,0.212
1947,Zimbabwe,2019,2.694,7.950,0.759,56.2,0.632,-0.064,0.831,0.716,0.235
1948,Zimbabwe,2020,3.160,7.829,0.717,56.8,0.643,-0.009,0.789,0.703,0.346


# Configurando a Conexão com o Banco de Dados (Neo4j)

In [187]:
driver = GraphDatabase.driver(
  neo_bolt_url,
  auth=basic_auth(neo_username, neo_password))

# Criando Funções Para Executar as Queries no Banco de Dados

In [188]:
def data_query_read(cypher_query):
    with driver.session(database="neo4j") as session:
        results = session.read_transaction(
            lambda tx: tx.run(cypher_query).data())

    driver.close()
    return results

In [189]:
def data_query_write(cypher_query):
    with driver.session(database="neo4j") as session:
        results = session.write_transaction(
            lambda tx: tx.run(cypher_query).data())

    driver.close()
    return results

# Executando Query de Leitura para Testar a Comunicação com o Banco de Dados

In [190]:
# Conta o total de registros que existem no banco de dados
cypher_query = '''
MATCH (n)
RETURN COUNT(n) AS count
LIMIT 10
'''

results = data_query_read(cypher_query)
for record in results:
        print(record['count'])

0


# Executando Query de Escrita para Testar a Comunicação com o Banco de Dados

In [191]:
# Cria registro teste
cypher_query = '''
CREATE (p:Person {name: 'John Doe', age: 30}) 
RETURN p
'''

results = data_query_write(cypher_query)
for record in results:
        print(record)

{'p': {'name': 'John Doe', 'age': 30}}


In [192]:
# Exclui o registro teste
cypher_query = '''
MATCH (p:Person {name: 'John Doe', age: 30}) 
DELETE p
'''

results = data_query_write(cypher_query)
for record in results:
        print(record)

In [193]:
# Exclui todos os nodes do banco de dados
cypher_query = '''
MATCH (n) 
DELETE n
'''

results = data_query_write(cypher_query)
for record in results:
        print(record)

# Função Que Trata os Valores Nulos

In [194]:
def convert_nan(value):
    if pd.isna(value):
        return 0
    return value

# Função Que Formata a String para a Query de Inserção de Registro

In [195]:
def format_cypher(row):
    format_row = f'''
        (:Happiness {{
         country_name: '{convert_nan(row['Country name'])}',
         year: {convert_nan(row['year'])},
         life_ladder: {convert_nan(row['Life Ladder'])},
         log_gdp_per_capita: {convert_nan(row['Log GDP per capita'])},
         social_support: {convert_nan(row['Social support'])},
         healthy_life_expectancy_at_birth: {convert_nan(row['Healthy life expectancy at birth'])},
         freedom_to_make_life_choices: {convert_nan(row['Freedom to make life choices'])},
         generosity: {convert_nan(row['Generosity'])},
         perceptions_of_corruption: {convert_nan(row['Perceptions of corruption'])},
         positive_affect: {convert_nan(row['Positive affect'])},
         negative_affect: {convert_nan(row['Negative affect'])}
      }})
      '''
    return format_row

# Importando os Dados do Arquivo CSV no Banco de Dados 

In [197]:
# Cria sub dataframe com os primeiros 500 registros do dataframe principal para fins didáticos
df_happiness_sub = df_happiness.iloc[0:500]

In [198]:
step = 50 # Quantidade de registros que será gravado no banco de dados a cada chamada (batch input)
total = len(df_happiness_sub.index)

for cont in range(0, total, step):
   
   df_happiness_sub_part = df_happiness_sub.iloc[cont:cont + step]
   
   # Apresenta indicadores do processamento
   print(f'''
   start-index: {cont}
   end-index: {cont + step}
   total-registros-sub: {len(df_happiness_sub_part)}
   total-registros-csv: {total}
   ''')

   cypher_rows = ''
   cont2 = 1

   # Formata as linhas no padrão cypher
   for index, row in df_happiness_sub_part.iterrows():
      
      # Tratativa para incluir a virgula ao final de cada registro
      if cont2 < len(df_happiness_sub_part):
         cypher_rows += f'{format_cypher(row)},'
      else:
         cypher_rows += f'{format_cypher(row)}'
      
      cont2 += 1

   # Formata a query de inserção de registros
   cypher_query = f'''
   CREATE {cypher_rows}
   '''

   #print(cypher_query)

   # Realiza inserção dos registros no banco de dados
   results = data_query_write(cypher_query)
   # for record in results:
   #    print(record)


   start-index: 0
   end-index: 50
   total-registros-sub: 50
   total-registros-csv: 500
   

   start-index: 50
   end-index: 100
   total-registros-sub: 50
   total-registros-csv: 500
   

   start-index: 100
   end-index: 150
   total-registros-sub: 50
   total-registros-csv: 500
   

   start-index: 150
   end-index: 200
   total-registros-sub: 50
   total-registros-csv: 500
   

   start-index: 200
   end-index: 250
   total-registros-sub: 50
   total-registros-csv: 500
   

   start-index: 250
   end-index: 300
   total-registros-sub: 50
   total-registros-csv: 500
   

   start-index: 300
   end-index: 350
   total-registros-sub: 50
   total-registros-csv: 500
   

   start-index: 350
   end-index: 400
   total-registros-sub: 50
   total-registros-csv: 500
   

   start-index: 400
   end-index: 450
   total-registros-sub: 50
   total-registros-csv: 500
   

   start-index: 450
   end-index: 500
   total-registros-sub: 50
   total-registros-csv: 500
   


# Consultando a Quantidade de Registros que Foram Importados no Banco de Dados

In [199]:
cypher_query = '''
MATCH (n)
RETURN COUNT(n) AS count
'''

results = data_query_read(cypher_query)
for record in results:
        print(record['count'])

500


# Lista os 100 Primeiros Registros que Foram Importados no Banco de Dados

In [200]:
cypher_query = '''
MATCH (h:Happiness)
RETURN h
LIMIT 100
'''

results = data_query_read(cypher_query)
for record in results:
  print(record)

{'h': {'generosity': 0.168, 'freedom_to_make_life_choices': 0.718, 'life_ladder': 3.724, 'year': 2008, 'healthy_life_expectancy_at_birth': 50.8, 'country_name': 'Afghanistan', 'negative_affect': 0.258, 'social_support': 0.451, 'positive_affect': 0.518, 'log_gdp_per_capita': 7.37, 'perceptions_of_corruption': 0.882}}
{'h': {'generosity': 0.19, 'freedom_to_make_life_choices': 0.679, 'life_ladder': 4.402, 'year': 2009, 'healthy_life_expectancy_at_birth': 51.2, 'country_name': 'Afghanistan', 'negative_affect': 0.237, 'social_support': 0.552, 'positive_affect': 0.584, 'log_gdp_per_capita': 7.54, 'perceptions_of_corruption': 0.85}}
{'h': {'generosity': 0.121, 'freedom_to_make_life_choices': 0.6, 'life_ladder': 4.758, 'year': 2010, 'healthy_life_expectancy_at_birth': 51.6, 'country_name': 'Afghanistan', 'negative_affect': 0.275, 'social_support': 0.539, 'positive_affect': 0.618, 'log_gdp_per_capita': 7.647, 'perceptions_of_corruption': 0.707}}
{'h': {'generosity': 0.162, 'freedom_to_make_life